In [48]:
import pandas as pd
import torch
import pickle
from lstmmodel import LSTMModel
from kiwipiepy import Kiwi 
from torchtext.vocab import Vocab
import re

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

DICT_PATH = '/home/parking/ml/data/MiniProj/models'

MODEL_PATH = '/app/data/models/lstm/comp_review_model_select_final_last_0.9915.pth'
VOCAB_DICT_PATH = '/app/data/models/lstm/voab_442290_final2.pkl'

class LSTMSenti:
    def __init__(self):
        
        
        
        
        
        self.__kiwi = Kiwi()
        
        vacab_dict = pickle.load(open(VOCAB_DICT_PATH, 'rb'))
        self.__vocab = Vocab(vacab_dict)
        self.__vocab.set_default_index(self.__vocab['<unk>'])
        
        # 단어(클래스) 갯수
        N_CLASS = len(self.__vocab)
        # 임베딩 차원 수 (임베딩된 단어의 벡터 차원 수)
        EMBEDDING_SIZE = 128
        # LSTM 레이어 은닉층의 차원 수
        HIDDEN_SIZE = 10
        # LSTM 레이어 은닉층 각각의 레이어 갯수
        NUM_LAYERS = 3
        # LSTM 레이어 양방향 여부
        IS_BIDIRECTIONAL = True
        
        self.__threshold = 0.5
        
        self.__model = LSTMModel(
            n_class=N_CLASS,
            embedding_size=EMBEDDING_SIZE,
            hidden_size=HIDDEN_SIZE,
            num_layers=NUM_LAYERS,
            is_bidirectional=IS_BIDIRECTIONAL
        )
        self.__model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
        self.__model.eval()
        self.__model.to(DEVICE)
        
        pass
    
    def __pretreatment_and_tokenizer(self, text):
        pattern = r'[^a-z가-힣\s]'
        text = re.sub(pattern=pattern, repl='', string=text)
        text = text.strip()
        
        doc = self.__kiwi.tokenize(text)
        return [token.form for token in doc]
    
    def __text_to_tensor(self, text):
        text = self.__pretreatment_and_tokenizer(text)
        if len(text) == 0:
            return None
        text = [self.__vocab[token] for token in text]
        text = torch.tensor(text).unsqueeze(0).to(DEVICE)
        return text
    
    def get_sentiment(self, text):
        text = self.__text_to_tensor(text)
        if text is None:
            return 'N'
        
        output = self.__model(text)
        # 이미 sigmoid를 거쳤기 때문에 sigmoid를 거치지 않음
        # output = torch.sigmoid(output)
        
        if output.item() > self.__threshold:
            return 'P'
        else:
            return 'N'

In [49]:
lstm_senti_model = LSTMSenti()

In [50]:
review = '이 회사 좀 많이 별로인거 같아요.'

In [51]:
lstm_senti_model.get_sentiment(review)

'N'

In [52]:
review = '휴가도 많이 주고 급여도 잘 주는 회사입니다.'

In [53]:
lstm_senti_model.get_sentiment(review)

'P'

In [54]:
review = 'good.'

In [55]:
lstm_senti_model.get_sentiment(review)

'N'

In [56]:
reviews = [
    '이 회사 좀 많이 별로인거 같아요.',
    '휴가도 많이 주고 급여도 잘 주는 회사입니다.',
    'good.',
    '이 회사 진짜 좋은거 같아요.',
    '이 회사 진짜 좋은거 같아요. 휴가도 많이 주고 급여도 잘 주는 회사입니다.',
    '이 회사 진짜 좋은거 같아요. 휴가도 많이 주고 급여도 잘 주는 회사입니다. good.',
    ' ',
]

df = pd.DataFrame(reviews, columns=['review'])
df

,review
0,이 회사 좀 많이 별로인거 같아요.
1,휴가도 많이 주고 급여도 잘 주는 회사입니다.
2,good.
3,이 회사 진짜 좋은거 같아요.
4,이 회사 진짜 좋은거 같아요. 휴가도 많이 주고 급여도 잘 주는 회사입니다.
5,이 회사 진짜 좋은거 같아요. 휴가도 많이 주고 급여도 잘 주는 회사입니다. good.
6,


In [57]:
df['senti'] = df['review'].apply(lstm_senti_model.get_sentiment)

In [58]:
df

,review,senti
0,이 회사 좀 많이 별로인거 같아요.,N
1,휴가도 많이 주고 급여도 잘 주는 회사입니다.,P
2,good.,N
3,이 회사 진짜 좋은거 같아요.,P
4,이 회사 진짜 좋은거 같아요. 휴가도 많이 주고 급여도 잘 주는 회사입니다.,P
5,이 회사 진짜 좋은거 같아요. 휴가도 많이 주고 급여도 잘 주는 회사입니다. good.,P
6,,N


In [75]:
from service_models import ServiceModels
from tqdm import tqdm, tqdm_pandas

In [76]:
torch_models = ServiceModels(type='reviews_senti')

In [78]:
tqdm_pandas(tqdm())

0it [00:00, ?it/s]


In [80]:
df['senti'] = df['review'].progress_apply(torch_models.get_reviews_sentiment)

7it [00:18,  2.71s/it]


In [72]:
df

,review,senti
0,이 회사 좀 많이 별로인거 같아요.,N
1,휴가도 많이 주고 급여도 잘 주는 회사입니다.,P
2,good.,N
3,이 회사 진짜 좋은거 같아요.,P
4,이 회사 진짜 좋은거 같아요. 휴가도 많이 주고 급여도 잘 주는 회사입니다.,P
5,이 회사 진짜 좋은거 같아요. 휴가도 많이 주고 급여도 잘 주는 회사입니다. good.,P
6,,N


In [65]:
pd.__version__

'2.0.3'

In [66]:
import tqdm
tqdm.__version__

'4.66.1'

In [ ]:
from tqdm import tqdm, tqdm_pandas